In [9]:
import os

from dotenv import load_dotenv, find_dotenv
_=load_dotenv(find_dotenv())

In [38]:
# old
#import openai
#openai.api_key = os.environ['OPENAI_API_KEY']

# new
from openai import OpenAI

client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY'],  # this is also the default, it can be omitted
)

In [39]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0613"

In [12]:
current_date

datetime.date(2024, 3, 4)

# Chat API
#### Direct API calls to OpenAI

In [40]:
def get_completion(prompt,model=llm_model):
    messages= [{"role":"user", "content":prompt}]
    response=client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

In [16]:
get_completion("What is 1+1?")

'1+1 equals 2.'

In [17]:
get_completion("What is Newton third law?")

"Newton's third law of motion states that for every action, there is an equal and opposite reaction. This means that whenever an object exerts a force on another object, the second object exerts an equal and opposite force back on the first object. In simpler terms, it means that forces always occur in pairs."

In [18]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [19]:
style= """ Provide a respectful and calm english tone"""

In [20]:
prompt= f""" Translate the text that is delimited by triple backticks into a style that is {style}.\
text: ```{customer_email}```"""


In [21]:
get_completion(prompt)

'I am quite frustrated that my blender lid flew off and made a mess of my kitchen walls with smoothie! To add to my troubles, the warranty does not cover the expenses of cleaning up my kitchen. I kindly request your assistance at this moment, my friend.'

## Using Langchain

In [41]:
#from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI

### Model

In [25]:
# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
chat = ChatOpenAI(temperature=0.0, model=llm_model)
chat

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7f1ac09f0dc0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f1ac09f2230>, model_name='gpt-3.5-turbo-0613', temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')

### Prompt Template

##### Example 1

In [27]:
template_string="""Transalte the text that is delimited by triple backtick into a style : {style}.\
text:```{text}``` """

In [26]:
from langchain.prompts import ChatPromptTemplate

In [29]:
prompt_template=ChatPromptTemplate.from_template(template_string)
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], template='Transalte the text that is delimited by triple backtick into a style : {style}.text:```{text}``` ')

In [30]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [31]:
customer_style = """American English \
in a calm and respectful tone
"""

In [32]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [33]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [34]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [35]:
print(customer_messages[0])

content="Transalte the text that is delimited by triple backtick into a style : American English in a calm and respectful tone\n.text:```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n``` "


In [42]:
# Call the LLM to translate to the style of the customer message
customer_response = chat.invoke(customer_messages)

In [43]:
customer_response

AIMessage(content='I am quite frustrated that my blender lid unexpectedly flew off and caused a messy splatter of smoothie all over my kitchen walls. To add to my dismay, the warranty does not cover the expenses for cleaning up my kitchen. I kindly request your assistance at this moment, my friend.')

In [44]:
print(customer_response.content)

I am quite frustrated that my blender lid unexpectedly flew off and caused a messy splatter of smoothie all over my kitchen walls. To add to my dismay, the warranty does not cover the expenses for cleaning up my kitchen. I kindly request your assistance at this moment, my friend.


##### Example 2

In [47]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. Tough luck! See ya! """

In [49]:
service_style_gentle= """ convert the input into a polite English service tone"""

In [51]:
service_message= prompt_template.format_messages(style=service_style_gentle, text=service_reply)

In [54]:
service_response=chat.invoke(service_message)

In [55]:
print(service_response.content)

Hey there customer, we regret to inform you that the warranty does not cover the expenses related to cleaning your kitchen. This is due to the fact that it appears you may have misused your blender by forgetting to put the lid on before starting it. We understand this may be disappointing news. Thank you for your understanding and have a great day!


## Output Parsers

Let's start with defining how we would like the LLM output to look like:

In [56]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [57]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""


review_template=""" For the following text, extract the following information:

gift: Was the item purchased as a gift for someone. If yes, keep this True else False.

delivery_days: If the delivery duration information is available keep this as the value avaialbel else keep it as -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

Then format the above infomration as a JSON output with following keys:
gift
delivery_days
price_value
text: {text} """

In [62]:
review_prompt_template=ChatPromptTemplate.from_template(review_template)

In [65]:
review_message=review_prompt_template.format_messages(text=customer_review)

In [69]:
response=chat.invoke(review_message)

In [70]:
print(response.content)

{
  "gift": false,
  "delivery_days": 2,
  "price_value": ["It's slightly more expensive than the other leaf blowers out there"],
  "text": "This leaf blower is pretty amazing. It has four settings: candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}


In [71]:
type(response.content)

str

### Parse the LLM output string into a Python dictionary
We will now format the output by giving specifi foramt instructions using below libraries

In [72]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [75]:
gift_schema= ResponseSchema(name="gift", description="Was the item purchased  as a gift for someone else? \
                             Answer True if yes,False if not or unknown.")


delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schema= [gift_schema, delivery_days_schema, price_value_schema]

In [78]:
output_parser=StructuredOutputParser.from_response_schemas(response_schema)

In [80]:
format_instructions = output_parser.get_format_instructions()

In [82]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased  as a gift for someone else?                              Answer True if yes,False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [83]:
review_template_with_format=""" For the following text, extract the following information:

gift: Was the item purchased as a gift for someone. If yes, keep this True else False.

delivery_days: If the delivery duration information is available keep this as the value avaialbel else keep it as -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

Then format the above infomration as a JSON output with following keys:
gift
delivery_days
price_value
text: {text} 
{format_instructions} """

In [85]:
review_prompt_template_2=ChatPromptTemplate.from_template(review_template_with_format)
review_message_2=review_prompt_template_2.format_messages(text=customer_review, format_instructions=format_instructions)


In [88]:
review_message_2[0].content

' For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone. If yes, keep this True else False.\n\ndelivery_days: If the delivery duration information is available keep this as the value avaialbel else keep it as -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nThen format the above infomration as a JSON output with following keys:\ngift\ndelivery_days\nprice_value\ntext: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife\'s anniversary present. I think my wife liked it so much she was speechless. So far I\'ve been the only one using it, and I\'ve been using it every other morning to clear the leaves on our lawn. It\'s slightly more expensive than the other leaf blowers out there, but I think it\'s worth it for the extra features.\n \nThe output should 

In [89]:
response=chat.invoke(review_message_2)

In [91]:
print(response.content)

```json
{
	"gift": false,
	"delivery_days": "2",
	"price_value": "It's slightly more expensive than the other leaf blowers out there"
}
```


In [92]:
output_dict = output_parser.parse(response.content)

In [93]:
output_dict

{'gift': False,
 'delivery_days': '2',
 'price_value': "It's slightly more expensive than the other leaf blowers out there"}

In [94]:
type(output_dict)

dict

In [95]:
output_dict.get('price_value')

"It's slightly more expensive than the other leaf blowers out there"